<a href="https://colab.research.google.com/github/nhgowtham/hackathon_gg/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preprocessing

### Dataset STO

In [ ]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tifffile import imread

# === Settings ===
ROOT_DIR = "/home/gowthamnh/hackathon_gg/data/image_data"
DATASETS = {
    "cdte_dataset.h5": "DataSet_CdTe",
    "sto_dataset.h5": "DataSet_STO"
}
N_PREVIEW = 5  # For visual inspection

# === Utils ===
def normalize_percentile(img, pmin=2, pmax=98):
    lo, hi = np.percentile(img, (pmin, pmax))
    return np.clip((img - lo) / (hi - lo), 0, 1)

def extract_label(fname):
    return os.path.basename(fname).split("_")[0].lower()

def find_all_tif_files(folder):
    tif_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".tif"):
                tif_files.append(os.path.join(root, file))
    return tif_files

# === Core Processing ===
def process_dataset(out_h5, subfolder):
    input_dir = os.path.join(ROOT_DIR, subfolder)
    all_files = find_all_tif_files(input_dir)
    print(f"📁 Processing {subfolder} ({len(all_files)} images)")

    label_map = {}
    key_list = []
    previews = []

    with h5py.File(out_h5, "w") as f:
        image_group = f.create_group("images")
        label_ds = f.create_group("labels")

        for path in all_files:
            try:
                raw = imread(path)
                if raw.ndim > 2:
                    raw = raw[..., 0]
                raw = np.squeeze(raw).astype(np.float32)
                norm = normalize_percentile(raw)

                fname = os.path.basename(path)
                label = extract_label(fname)
                key = os.path.splitext(fname)[0].lower()

                # Avoid duplicates
                if key in image_group:
                    i = 1
                    while f"{key}_{i}" in image_group:
                        i += 1
                    key = f"{key}_{i}"

                image_group.create_dataset(key, data=norm, compression="gzip")
                label_ds.attrs[key] = label
                key_list.append(key)

                label_map[label] = label_map.get(label, 0) + 1

                if len(previews) < N_PREVIEW:
                    previews.append((raw, norm, label, fname))

            except Exception as e:
                print(f"[ERROR] {path}: {e}")

        f.create_dataset("image_keys", data=np.array(key_list, dtype='S'))

    print(f"✅ Saved to {out_h5}")
    print("📊 Label summary:", label_map)
    return previews

# === Plot ===
def plot_previews(previews, title):
    fig, axs = plt.subplots(len(previews), 2, figsize=(6, 2 * len(previews)))
    for i, (raw, norm, label, fname) in enumerate(previews):
        axs[i, 0].imshow(raw, cmap='viridis')
        axs[i, 0].set_title(f"Original - {fname}")
        axs[i, 1].imshow(norm, cmap='viridis')
        axs[i, 1].set_title(f"Normalized - label: {label}")
    fig.suptitle(title)
    plt.tight_layout()
    plt.show()

# === Execute ===
for out_file, folder in DATASETS.items():
    previews = process_dataset(out_file, folder)
    plot_previews(previews, f"Preview from {out_file}")


In [44]:
import pathlib, re, h5py, numpy as np, tifffile as tiff
from tqdm import tqdm

sto_root = pathlib.Path("data/image_data/DataSet_STO")  # 64x64 tif patches; label prefix in filename
h5_out = pathlib.Path("sto_dataset.h5")
label_regex = re.compile(r"^([A-Za-z0-9]+)_")  # prefix before first underscore

def collect_files(root: pathlib.Path):
    files = sorted([p for p in root.rglob("*.tif")])
    recs = []
    for p in files:
        m = label_regex.match(p.name)
        if not m:
            continue
        recs.append({"path": p, "label": m.group(1)})
    return recs

records = collect_files(sto_root)
labels = sorted({r["label"] for r in records})
label_to_id = {l: i for i, l in enumerate(labels)}
print(f"found {len(records)} files; labels={labels}")

with h5py.File(h5_out, "w") as f:
    images_ds = f.create_dataset("images", shape=(len(records), 64, 64), dtype="float32")
    labels_ds = f.create_dataset("labels", shape=(len(records),), dtype="int32")
    paths_ds = f.create_dataset("meta/paths", shape=(len(records),), dtype=h5py.string_dtype())
    label_names_ds = f.create_dataset("meta/label_names", data=np.array(labels, dtype="S"))
    for i, rec in enumerate(tqdm(records)):
        img = tiff.imread(rec["path"])
        if img.ndim == 3:
            img = img[..., 0]
        images_ds[i] = img.astype("float32")
        labels_ds[i] = label_to_id[rec["label"]]
        paths_ds[i] = str(rec["path"].relative_to(sto_root))
print("wrote", h5_out)



found 764 files; labels=['Bulk', 'Extra', 'IExtra', 'Mis']


100%|██████████| 764/764 [00:00<00:00, 2087.06it/s]

wrote sto_dataset.h5


# Atom finding

In [45]:
import os
import h5py, numpy as np, matplotlib.pyplot as plt
from skimage.restoration import denoise_tv_chambolle
from skimage.filters import difference_of_gaussians
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy.spatial import cKDTree

# =========================
# CONFIG
# =========================
H5_IN = "sto_dataset.h5"
OUT_DIR = "mask_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# indices you requested
CASE_IDXS = [0, 300, 500, 762]

# if you want label-aware interstitial gating, set this True
USE_LABELS_FROM_H5 = True

# base params (use the same defaults you had)
denoise_weight = 0.30
dog_sigma1, dog_sigma2 = 0.6, 3.0
peak_min_dist = 3
peak_thr_k = 0.5

use_watershed = True

# IMPORTANT: keep assign_radius, but also provide an option to auto-scale from lattice spacing
USE_AUTO_ASSIGN = True         # recommended
ASSIGN_MULT = 0.45             # r_assign = ASSIGN_MULT * lattice_spacing
assign_radius_fallback = 10.0  # used only if auto fails

fft_peak_exclude = 3
fft_topk = 12
fft_min_dist = 4
fft_collinear_thresh = 0.93

# semantic IDs
ID2NAME = {0:"bulk", 1:"missing", 2:"extra_atom", 3:"extra_interstitial"}
NAME2ID = {v:k for k,v in ID2NAME.items()}

COLOR = {
    "bulk":              np.array([0.0, 0.0, 0.0, 0.0]),
    "missing":           np.array([0.2, 0.45, 1.0, 0.40]),
    "extra_atom":        np.array([1.0, 0.1, 0.1, 0.55]),
    "extra_interstitial":np.array([1.0, 0.9, 0.1, 0.55]),
}

# =========================
# HELPERS
# =========================
def normalize01(img):
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, (1, 99))
    return np.clip((img - p1) / (p99 - p1 + 1e-6), 0, 1)

def locate_atoms(img, denoise_weight, dog_sigma1, dog_sigma2, peak_min_dist, peak_thr_k, use_watershed):
    img = img.astype(np.float32)
    imgz = (img - img.mean()) / (img.std() + 1e-6)

    den = denoise_tv_chambolle(imgz, weight=denoise_weight)
    band = difference_of_gaussians(den, dog_sigma1, dog_sigma2)

    thr = float(band.mean() + peak_thr_k * band.std())
    coords = peak_local_max(
        band,
        min_distance=peak_min_dist,
        threshold_abs=thr,
        exclude_border=False,
    ).astype(np.int32)

    labels = None
    if use_watershed and len(coords) > 0:
        markers = np.zeros_like(img, dtype=np.int32)
        for i, (r, c) in enumerate(coords, 1):
            markers[r, c] = i
        labels = watershed(-band, markers, mask=(band > band.mean()))

    return {
        "raw": img,
        "norm01": normalize01(img),
        "denoise": den,
        "band": band,
        "coords": coords,
        "labels": labels,
        "thr": thr,
    }

def estimate_lattice_fft_robust(img, topk=24, exclude=3, min_dist=4, collinear_thresh=0.93):
    fft = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    cy, cx = np.array(fft.shape) // 2

    rr, cc = np.ogrid[:fft.shape[0], :fft.shape[1]]
    fft[(rr - cy) ** 2 + (cc - cx) ** 2 <= exclude ** 2] = 0

    rec_peaks = peak_local_max(fft, min_distance=min_dist, threshold_abs=0, exclude_border=False)
    if len(rec_peaks) < 6:
        raise RuntimeError("Too few FFT peaks found")

    vals = fft[rec_peaks[:, 0], rec_peaks[:, 1]]
    order = np.argsort(vals)[::-1][:topk]
    rec_peaks = rec_peaks[order]

    cand = []
    for p in rec_peaks:
        delta = (p - np.array([cy, cx], dtype=np.float32))
        if np.allclose(delta, 0):
            continue
        freq_vec = delta / np.array(img.shape, dtype=np.float32)
        norm = np.linalg.norm(freq_vec)
        if norm < 1e-6:
            continue
        real_vec = (freq_vec / norm) * (1.0 / norm)
        cand.append(real_vec.astype(np.float32))

    cand = np.stack(cand, axis=0)

    best = None
    best_score = -np.inf
    for i in range(len(cand)):
        for j in range(i+1, len(cand)):
            v1, v2 = cand[i], cand[j]
            n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
            if n1 < 1e-3 or n2 < 1e-3:
                continue
            cosang = np.abs(np.dot(v1, v2) / (n1*n2 + 1e-12))
            if cosang > collinear_thresh:
                continue
            score = (1.0 - cosang) - 0.25*np.abs(np.log((n1+1e-6)/(n2+1e-6)))
            if score > best_score:
                best_score = score
                best = (v1, v2)

    if best is None:
        raise RuntimeError("FFT basis selection failed")
    return best

def lattice_spacing(v1, v2):
    return float(0.5 * (np.linalg.norm(v1) + np.linalg.norm(v2)))

def generate_lattice_sites(img_shape, origin, v1, v2):
    h, w = img_shape
    A = np.stack([v1, v2], axis=1).astype(np.float32)
    Ai = np.linalg.pinv(A)

    corners = np.array([[0,0],[0,w],[h,0],[h,w]], dtype=np.float32) - origin
    uv = corners @ Ai.T

    umin, vmin = np.floor(uv.min(axis=0) - 2).astype(int)
    umax, vmax = np.ceil(uv.max(axis=0) + 2).astype(int)

    pts = []
    for u in range(umin, umax+1):
        for v in range(vmin, vmax+1):
            p = origin + u*v1 + v*v2
            if 0 <= p[0] < h and 0 <= p[1] < w:
                pts.append(p)
    return np.array(pts, dtype=np.float32)

def refine_origin_unitcell(peaks, origin, v1, v2, img_shape, tol, grid=21):
    H, W = img_shape
    peaks = peaks.astype(np.float32)
    if len(peaks) < 6:
        return origin.astype(np.float32), 0

    treeP = cKDTree(peaks)
    A = np.stack([v1, v2], axis=1).astype(np.float32)
    Ai = np.linalg.pinv(A)
    corners = np.array([[0,0],[0,W],[H,0],[H,W]], dtype=np.float32)

    def expected_for_origin(o):
        uv = (corners - o) @ Ai.T
        umin, vmin = np.floor(uv.min(axis=0) - 2).astype(int)
        umax, vmax = np.ceil(uv.max(axis=0) + 2).astype(int)
        pts = []
        for u in range(umin, umax+1):
            for v in range(vmin, vmax+1):
                p = o + u*v1 + v*v2
                if 0 <= p[0] < H and 0 <= p[1] < W:
                    pts.append(p)
        return np.array(pts, dtype=np.float32)

    best_score, best_origin = -1, origin.astype(np.float32)
    aa = np.linspace(0.0, 1.0, grid, endpoint=False)
    bb = np.linspace(0.0, 1.0, grid, endpoint=False)

    for a in aa:
        for b in bb:
            o = origin + a*v1 + b*v2
            exp_sites = expected_for_origin(o)
            if len(exp_sites) == 0:
                continue
            d, _ = treeP.query(exp_sites, k=1)
            score = int(np.sum(d <= tol))
            if score > best_score:
                best_score, best_origin = score, o

    return best_origin, best_score

def assign_and_classify(detected, expected, r_assign):
    detected = detected.astype(np.float32)
    expected = expected.astype(np.float32)

    if len(expected) == 0:
        return (np.array([], dtype=np.uint8),
                np.array([], dtype=np.int32),
                np.zeros((0,2), dtype=np.int32),
                np.zeros((0,2), dtype=np.int32))

    treeE = cKDTree(expected)
    d_dist, d_nn = treeE.query(detected, k=1)
    close = d_dist <= r_assign

    exp_to_det = -np.ones(len(expected), dtype=np.int32)
    by_site = {}
    for j in range(len(detected)):
        if close[j]:
            by_site.setdefault(int(d_nn[j]), []).append(j)

    extra_atom = []
    for sid, js in by_site.items():
        js = np.array(js, dtype=int)
        j0 = js[np.argmin(d_dist[js])]
        exp_to_det[sid] = j0
        for jj in js:
            if jj != j0:
                extra_atom.append(detected[jj])

    site_class = np.zeros(len(expected), dtype=np.uint8)
    site_class[exp_to_det < 0] = 1  # missing

    extra_atom = np.array(extra_atom, dtype=np.int32) if len(extra_atom) else np.zeros((0,2), dtype=np.int32)
    extra_interstitial = detected[~close].astype(np.int32) if len(detected) else np.zeros((0,2), dtype=np.int32)

    return site_class, exp_to_det, extra_atom, extra_interstitial

def semantic_mask(img_shape, expected, site_class, extra_atom, extra_interstitial, watershed_labels=None, stamp_radius=3):
    H, W = img_shape
    rr, cc = np.mgrid[0:H, 0:W]
    pix = np.stack([rr.ravel(), cc.ravel()], axis=1).astype(np.float32)

    treeE = cKDTree(expected.astype(np.float32))
    _, nn = treeE.query(pix, k=1)
    nn = nn.reshape(H, W)

    seg = np.zeros((H, W), dtype=np.uint8)
    seg[site_class[nn] == 1] = NAME2ID["missing"]

    if watershed_labels is not None:
        seg_ws = np.zeros_like(seg)
        nreg = int(watershed_labels.max())
        for rid in range(1, nreg+1):
            m = (watershed_labels == rid)
            if not np.any(m):
                continue
            ys, xs = np.where(m)
            cy, cx = float(ys.mean()), float(xs.mean())
            _, sid = treeE.query([[cy, cx]], k=1)
            sid = int(sid[0])
            seg_ws[m] = NAME2ID["missing"] if site_class[sid] == 1 else NAME2ID["bulk"]
        seg = seg_ws

    rad = int(max(2, stamp_radius))

    def stamp(points, cls_id):
        if len(points) == 0:
            return
        for (r, c) in points:
            r0, r1 = max(0, r-rad), min(H, r+rad+1)
            c0, c1 = max(0, c-rad), min(W, c+rad+1)
            seg[r0:r1, c0:c1] = np.maximum(seg[r0:r1, c0:c1], cls_id)

    stamp(extra_atom, NAME2ID["extra_atom"])
    stamp(extra_interstitial, NAME2ID["extra_interstitial"])
    return seg

def overlay_rgba(gray01, seg):
    rgba = np.dstack([gray01, gray01, gray01, np.ones_like(gray01)])
    out = rgba.copy()
    for name, col in COLOR.items():
        cls = NAME2ID[name]
        a = float(col[3])
        if a <= 0:
            continue
        m = (seg == cls)
        out[m] = (1-a)*out[m] + a*col
    return out

def save_debug_png(res, expected, site_class, exp_to_det, seg, out_path_prefix):
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    ax[0].imshow(res["band"], cmap="magma")
    ax[0].set_title(f"DoG bandpass (thr={res['thr']:.3f})")
    ax[0].axis("off")

    ax[1].imshow(overlay_rgba(res["norm01"], seg))
    ax[1].set_title("semantic overlay")
    ax[1].axis("off")

    plt.tight_layout()
    fig.savefig(out_path_prefix + "_dog_overlay.png", dpi=160)
    plt.close(fig)

# =========================
# MAIN
# =========================
with h5py.File(H5_IN, "r") as fin:
    imgs = fin["images"]
    labels = fin["labels"][:] if ("labels" in fin and USE_LABELS_FROM_H5) else None
    label_names = None
    if USE_LABELS_FROM_H5 and "meta/label_names" in fin:
        label_names = [x.decode() for x in fin["meta/label_names"][:]]

# output H5 with only 4 masks
h5_out = os.path.join(OUT_DIR, "sto_4case_masks.h5")
with h5py.File(H5_IN, "r") as fin, h5py.File(h5_out, "w") as fout:
    g = fout.create_group("cases")
    for idx in CASE_IDXS:
        img = fin["images"][idx]

        # decide allow_interstitial per-case from H5 label if possible
        allow_interstitial = False
        case_label_str = "unknown"
        if labels is not None and label_names is not None:
            case_label_str = label_names[int(labels[idx])]
            allow_interstitial = ("IExtra_3" in case_label_str)
        else:
            case_label_str = image_label
            allow_interstitial = ("IExtra_3" in case_label_str)

        res = locate_atoms(img, denoise_weight, dog_sigma1, dog_sigma2, peak_min_dist, peak_thr_k, use_watershed)

        v1, v2 = estimate_lattice_fft_robust(
            res["denoise"], topk=fft_topk, exclude=fft_peak_exclude, min_dist=fft_min_dist, collinear_thresh=fft_collinear_thresh
        )
        lat = lattice_spacing(v1, v2)

        # robust radius
        r_assign = ASSIGN_MULT * lat if USE_AUTO_ASSIGN else assign_radius_fallback
        r_assign = float(max(3.0, r_assign))

        if len(res["coords"]) > 0:
            origin0 = res["coords"][np.argmin(np.sum((res["coords"] - np.median(res["coords"], axis=0))**2, axis=1))].astype(np.float32)
        else:
            origin0 = (np.array(img.shape, dtype=np.float32) / 2.0)

        origin, _ = refine_origin_unitcell(
            peaks=res["coords"], origin=origin0, v1=v1, v2=v2, img_shape=img.shape, tol=r_assign, grid=21
        )

        expected = generate_lattice_sites(img.shape, origin, v1, v2)
        site_class, exp_to_det, extra_atom, extra_interstitial = assign_and_classify(res["coords"], expected, r_assign=r_assign)

        if not allow_interstitial:
            extra_interstitial = np.zeros((0,2), dtype=np.int32)

        seg = semantic_mask(
            img.shape, expected, site_class, extra_atom, extra_interstitial,
            watershed_labels=res["labels"] if use_watershed else None,
            stamp_radius=max(2, int(0.18 * lat))
        )

        # save to h5
        cg = g.create_group(str(idx))
        cg.create_dataset("image", data=img, compression="gzip")
        cg.create_dataset("mask", data=seg.astype(np.uint8), compression="gzip")
        cg.create_dataset("coords_detected", data=res["coords"].astype(np.int32), compression="gzip")
        cg.create_dataset("sites_expected", data=expected.astype(np.float32), compression="gzip")
        cg.attrs["label_str"] = case_label_str
        cg.attrs["allow_interstitial"] = bool(allow_interstitial)
        cg.attrs["lat_spacing_px"] = float(lat)
        cg.attrs["r_assign_px"] = float(r_assign)
        cg.attrs["peak_thr_k"] = float(peak_thr_k)

        # save debug PNG
        prefix = os.path.join(OUT_DIR, f"idx{idx}_label{case_label_str.replace('/','_')}")
        save_debug_png(res, expected, site_class, exp_to_det, seg, prefix)

        print(f"[idx={idx}] label={case_label_str} lat={lat:.2f}px r_assign={r_assign:.2f}px "
              f"peaks={len(res['coords'])} expected={len(expected)} "
              f"missing_sites={int(np.sum(site_class==1))} extras={len(extra_atom)} inter={len(extra_interstitial)}")

print("Saved:", h5_out)
print("PNG debug outputs in:", OUT_DIR)


[idx=0] label=Bulk lat=15.08px r_assign=6.79px peaks=20 expected=18 missing_sites=0 extras=0 inter=0
[idx=300] label=Extra lat=15.08px r_assign=6.79px peaks=21 expected=18 missing_sites=0 extras=0 inter=0
[idx=500] label=IExtra lat=15.08px r_assign=6.79px peaks=20 expected=18 missing_sites=0 extras=2 inter=0
[idx=762] label=Mis lat=15.08px r_assign=6.79px peaks=21 expected=18 missing_sites=0 extras=0 inter=0
Saved: mask_outputs/sto_4case_masks.h5
PNG debug outputs in: mask_outputs


In [41]:
import h5py
import numpy as np

def _is_dataset(obj):
    return isinstance(obj, h5py.Dataset)

def _find_first_dataset_by_name(h5f, name_candidates):
    """
    Search entire file for dataset whose basename matches any candidate.
    Returns dataset path or None.
    """
    found = []

    def visitor(name, obj):
        if _is_dataset(obj):
            base = name.split("/")[-1]
            if base in name_candidates:
                found.append(name)

    h5f.visititems(visitor)
    return found[0] if found else None

def get_images_dataset(h5f):
    # common cases: "images" dataset or nested
    if "images" in h5f and _is_dataset(h5f["images"]):
        return h5f["images"]
    p = _find_first_dataset_by_name(h5f, {"images", "image"})
    if p is None:
        raise KeyError("Could not find images dataset in H5 (looked for 'images'/'image').")
    return h5f[p]

def get_labels_array(h5f, allow_missing=True):
    """
    Returns labels as numpy array or None if missing and allow_missing=True.
    Handles cases where 'labels' is a group.
    """
    # direct dataset
    if "labels" in h5f and _is_dataset(h5f["labels"]):
        return h5f["labels"][:]

    # if 'labels' is a group, try inside it
    if "labels" in h5f and isinstance(h5f["labels"], h5py.Group):
        grp = h5f["labels"]
        for k in ["labels", "y", "target", "class", "classes"]:
            if k in grp and _is_dataset(grp[k]):
                return grp[k][:]

    # global search
    p = _find_first_dataset_by_name(h5f, {"labels", "y", "target"})
    if p is not None:
        return h5f[p][:]

    if allow_missing:
        return None
    raise KeyError("Could not find labels dataset in H5.")

def get_label_names(h5f):
    """
    Returns list[str] or None.
    """
    # common path
    if "meta/label_names" in h5f and _is_dataset(h5f["meta/label_names"]):
        return [x.decode() if isinstance(x, (bytes, np.bytes_)) else str(x) for x in h5f["meta/label_names"][:]]

    # search
    p = _find_first_dataset_by_name(h5f, {"label_names", "names", "class_names"})
    if p is None:
        return None
    arr = h5f[p][:]
    return [x.decode() if isinstance(x, (bytes, np.bytes_)) else str(x) for x in arr]

def debug_print_h5_tree(h5_path, max_lines=80):
    """
    Prints a compact tree so you can see what 'labels' actually is.
    """
    lines = []
    with h5py.File(h5_path, "r") as f:
        def visitor(name, obj):
            if isinstance(obj, h5py.Group):
                lines.append(f"G  {name}")
            else:
                lines.append(f"D  {name}  shape={obj.shape} dtype={obj.dtype}")
        f.visititems(visitor)

    print("\n".join(lines[:max_lines]))
    if len(lines) > max_lines:
        print(f"... ({len(lines)-max_lines} more)")

debug_print_h5_tree("sto_dataset.h5", max_lines=120)


D  image_keys  shape=(764,) dtype=|S10
G  images
D  images/bulk_0  shape=(64, 64) dtype=float64
D  images/bulk_1  shape=(64, 64) dtype=float64
D  images/bulk_10  shape=(64, 64) dtype=float64
D  images/bulk_100  shape=(64, 64) dtype=float64
D  images/bulk_101  shape=(64, 64) dtype=float64
D  images/bulk_102  shape=(64, 64) dtype=float64
D  images/bulk_103  shape=(64, 64) dtype=float64
D  images/bulk_104  shape=(64, 64) dtype=float64
D  images/bulk_105  shape=(64, 64) dtype=float64
D  images/bulk_106  shape=(64, 64) dtype=float64
D  images/bulk_107  shape=(64, 64) dtype=float64
D  images/bulk_108  shape=(64, 64) dtype=float64
D  images/bulk_109  shape=(64, 64) dtype=float64
D  images/bulk_11  shape=(64, 64) dtype=float64
D  images/bulk_110  shape=(64, 64) dtype=float64
D  images/bulk_111  shape=(64, 64) dtype=float64
D  images/bulk_112  shape=(64, 64) dtype=float64
D  images/bulk_113  shape=(64, 64) dtype=float64
D  images/bulk_114  shape=(64, 64) dtype=float64
D  images/bulk_115  shape=

In [42]:
import h5py, numpy as np

h5_in = "sto_dataset.h5"
h5_mask_out = "sto_masks_level0.h5"

with h5py.File(h5_in, "r") as fin, h5py.File(h5_mask_out, "w") as fout:
    imgs = fin["images"]
    labels = fin["labels"][:] if "labels" in fin else None
    n = len(imgs)
    H, W = imgs[0].shape

    seg_ds = fout.create_dataset("seg", shape=(n, H, W), dtype="uint8", compression="gzip")
    fout.create_dataset("labels", data=labels) if labels is not None else None

    # carry metadata if available
    for k in ["meta/paths", "meta/label_names"]:
        if k in fin:
            fout.create_dataset(k, data=fin[k][:])

    for i in range(n):
        img = imgs[i]
        # ---- run your pipeline here ----
        res = locate_atoms(img)
        v1, v2 = estimate_lattice_fft_robust(
            res["denoise"],
            topk=fft_topk,
            exclude=fft_peak_exclude,
            min_dist=fft_min_dist,
            collinear_thresh=fft_collinear_thresh
        )
        origin0 = res["coords"][np.argmin(np.sum((res["coords"] - np.median(res["coords"], axis=0))**2, axis=1))].astype(np.float32) if len(res["coords"]) else (np.array(img.shape, dtype=np.float32)/2)
        origin, _ = refine_origin_unitcell(res["coords"], origin0, v1, v2, img.shape, tol=assign_radius, grid=31)
        expected = generate_lattice_sites(img.shape, origin, v1, v2)
        site_class, exp_to_det, extra_atom, extra_interstitial = assign_and_classify(res["coords"], expected, r_assign=assign_radius)

        # interstitial gating using true label name (if you have it)
        allow_interstitial = False
        if labels is not None and "meta/label_names" in fin:
            names = [x.decode() for x in fin["meta/label_names"][:]]
            allow_interstitial = ("IExtra_3" in names[int(labels[i])])
        if not allow_interstitial:
            extra_interstitial = np.zeros((0,2), dtype=np.int32)

        seg = semantic_mask(
            img.shape, expected, site_class, extra_atom, extra_interstitial,
            watershed_labels=res["labels"] if use_watershed else None,
            stamp_radius=assign_radius/3
        )
        seg_ds[i] = seg

print("Saved masks to", h5_mask_out)


TypeError: Accessing a group is done with bytes or str, not <class 'slice'>

# Mask free embedding

In [ ]:
# Jupyter cell: MASK-FREE DINOv2 embeddings using top-k patch pooling (no ROI/masks)
import h5py, numpy as np, torch
import torch.nn.functional as F
from PIL import Image

DINO_MODEL = "dinov2_vitb14"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PATCH_SIZE = 224
batch_size = 128

h5_in  = "sto_dataset.h5"
h5_out = "sto_embeddings_nomask.h5"

IMNET_MEAN = (0.485, 0.456, 0.406)
IMNET_STD  = (0.229, 0.224, 0.225)

# ---- load model ----
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()

def normalize01(img2d):
    img2d = img2d.astype(np.float32)
    p1, p99 = np.percentile(img2d, (1, 99))
    return np.clip((img2d - p1) / (p99 - p1 + 1e-6), 0, 1)

def pil_to_tensor_no_numpy(pil_img: Image.Image) -> torch.Tensor:
    pil_img = pil_img.convert("RGB")
    w, h = pil_img.size
    b = pil_img.tobytes()
    storage = torch.ByteStorage.from_buffer(b)
    x = torch.ByteTensor(storage).view(h, w, 3).permute(2, 0, 1).contiguous()
    return x.float().div_(255.0)

def preprocess_gray2d_to_dino(img2d, out_size=224):
    # full image -> uint8 RGB
    g01 = normalize01(img2d)
    u8 = (g01 * 255.0 + 0.5).astype(np.uint8)
    rgb = np.repeat(u8[..., None], 3, axis=-1)  # HxWx3

    h, w, _ = rgb.shape
    pil = Image.frombytes("RGB", (w, h), rgb.tobytes())
    x = pil_to_tensor_no_numpy(pil).unsqueeze(0)  # (1,3,H,W)

    x = F.interpolate(x, size=(out_size, out_size), mode="bicubic", align_corners=False)

    mean = torch.tensor(IMNET_MEAN, dtype=x.dtype).view(1, 3, 1, 1)
    std  = torch.tensor(IMNET_STD,  dtype=x.dtype).view(1, 3, 1, 1)
    x = (x - mean) / std
    return x

@torch.inference_mode()
def embed_batch_topk_torch(img2d_list, topk=32):
    xs = [preprocess_gray2d_to_dino(im, out_size=PATCH_SIZE) for im in img2d_list]
    x = torch.cat(xs, dim=0).to(DEVICE, non_blocking=True)  # (B,3,224,224)

    feats = model.forward_features(x)["x_norm_patchtokens"]  # (B, N, D)
    score = torch.linalg.norm(feats, dim=-1)                # (B, N)
    k = min(topk, feats.shape[1])
    idx = torch.topk(score, k=k, dim=1, largest=True).indices  # (B, k)

    B, N, D = feats.shape
    idx_exp = idx.unsqueeze(-1).expand(B, k, D)
    top_tokens = torch.gather(feats, dim=1, index=idx_exp)  # (B,k,D)
    emb = top_tokens.mean(dim=1)                            # (B,D)

    return emb.detach().to("cpu", non_blocking=False).float().contiguous()


# ---- main ----
with h5py.File(h5_in, "r") as fin, h5py.File(h5_out, "w") as fout:
    imgs = fin["images"]
    labels = fin["labels"][:] if "labels" in fin else None
    n = len(imgs)

    emb_dim = 768  # vitb14
    emb_ds = fout.create_dataset("embeddings", shape=(n, emb_dim), dtype="float32", compression="gzip")
    if labels is not None:
        fout.create_dataset("labels", data=labels)

    for k in ["meta/paths", "meta/label_names"]:
        if k in fin:
            fout.create_dataset(k, data=fin[k][:])

    buf, idxs = [], []
    for i in range(n):
        buf.append(imgs[i])
        idxs.append(i)
        if len(buf) == batch_size or i == n - 1:
            emb_t = embed_batch_topk_torch(buf, topk=32)        # torch CPU tensor (B,D)
            emb_np = np.asarray(emb_t.tolist(), dtype=np.float32)  # slow-ish but works without torch<->numpy bridge
            emb_ds[idxs[0]:idxs[0] + len(buf)] = emb_np
            buf, idxs = [], []

print("Saved mask-free embeddings to", h5_out)


In [40]:
# Jupyter cell: MASK-FREE DINOv2 embeddings using top-k patch pooling (no ROI/masks)
import h5py, numpy as np, torch
import torch.nn.functional as F
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DINO_MODEL = "dinov2_vitb14"
PATCH_SIZE = 224
batch_size = 128

h5_in  = "sto_dataset.h5"
h5_out = "sto_embeddings_nomask.h5"

IMNET_MEAN = (0.485, 0.456, 0.406)
IMNET_STD  = (0.229, 0.224, 0.225)

# ---- load model ----
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()

def normalize01(img2d):
    img2d = img2d.astype(np.float32)
    p1, p99 = np.percentile(img2d, (1, 99))
    return np.clip((img2d - p1) / (p99 - p1 + 1e-6), 0, 1)

def pil_to_tensor_no_numpy(pil_img: Image.Image) -> torch.Tensor:
    pil_img = pil_img.convert("RGB")
    w, h = pil_img.size
    b = pil_img.tobytes()
    storage = torch.ByteStorage.from_buffer(b)
    x = torch.ByteTensor(storage).view(h, w, 3).permute(2, 0, 1).contiguous()
    return x.float().div_(255.0)

def preprocess_gray2d_to_dino(img2d, out_size=224):
    g01 = normalize01(img2d)
    u8 = (g01 * 255.0 + 0.5).astype(np.uint8)
    rgb = np.repeat(u8[..., None], 3, axis=-1)  # HxWx3

    h, w, _ = rgb.shape
    pil = Image.frombytes("RGB", (w, h), rgb.tobytes())
    x = pil_to_tensor_no_numpy(pil).unsqueeze(0)  # (1,3,H,W)
    x = F.interpolate(x, size=(out_size, out_size), mode="bicubic", align_corners=False)

    mean = torch.tensor(IMNET_MEAN, dtype=x.dtype).view(1, 3, 1, 1)
    std  = torch.tensor(IMNET_STD,  dtype=x.dtype).view(1, 3, 1, 1)
    return (x - mean) / std

@torch.inference_mode()
def embed_batch_topk_torch(img2d_list, topk=32):
    xs = [preprocess_gray2d_to_dino(im, out_size=PATCH_SIZE) for im in img2d_list]
    x = torch.cat(xs, dim=0).to(DEVICE, non_blocking=True)  # (B,3,224,224)

    feats = model.forward_features(x)["x_norm_patchtokens"]  # (B, N, D)
    score = torch.linalg.norm(feats, dim=-1)                # (B, N)
    k = min(topk, feats.shape[1])
    idx = torch.topk(score, k=k, dim=1, largest=True).indices  # (B, k)

    B, N, D = feats.shape
    idx_exp = idx.unsqueeze(-1).expand(B, k, D)
    top_tokens = torch.gather(feats, dim=1, index=idx_exp)  # (B,k,D)
    emb = top_tokens.mean(dim=1)                            # (B,D)
    return emb.detach().to("cpu", non_blocking=False).float().contiguous()

# ---- main ----
with h5py.File(h5_in, "r") as fin, h5py.File(h5_out, "w") as fout:
    imgs = get_images_dataset(fin)
    labels = get_labels_array(fin, allow_missing=True)
    label_names = get_label_names(fin)

    n = len(imgs)
    emb_dim = 768
    emb_ds = fout.create_dataset("embeddings", shape=(n, emb_dim), dtype="float32", compression="gzip")

    if labels is not None:
        fout.create_dataset("labels", data=labels)
    if label_names is not None:
        fout.create_dataset("meta/label_names", data=np.array(label_names, dtype="S"))

    # copy other helpful metadata if present
    for k in ["meta/paths", "paths"]:
        if k in fin and isinstance(fin[k], h5py.Dataset):
            fout.create_dataset("meta/paths", data=fin[k][:])

    buf, idxs = [], []
    for i in range(n):
        buf.append(imgs[i])
        idxs.append(i)
        if len(buf) == batch_size or i == n - 1:
            emb_t = embed_batch_topk_torch(buf, topk=32)  # torch CPU (B,D)
            emb_np = np.asarray(emb_t.tolist(), dtype=np.float32)  # bridge-safe
            emb_ds[idxs[0]:idxs[0] + len(buf)] = emb_np
            buf, idxs = [], []

print("Saved mask-free embeddings to", h5_out)


Using cache found in /home/gowthamnh/.cache/torch/hub/facebookresearch_dinov2_main


KeyError: "Could not find images dataset in H5 (looked for 'images'/'image')."

### Self supervised embedding

In [ ]:
# Jupyter cell: ROI-based DINOv2 embeddings WITHOUT torchvision ToTensor / torch.from_numpy
# Works around: RuntimeError: Numpy is not available

import h5py, numpy as np, torch
import torch.nn.functional as F
from PIL import Image

# ----------------------------
# Config
# ----------------------------
DINO_MODEL = "dinov2_vitb14"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PATCH_SIZE = 224

h5_in = "sto_dataset.h5"
h5_masks = "sto_masks_level0.h5"      # produced by your Level-0 pipeline
h5_out = "sto_embeddings_roi.h5"
batch_size = 128

# ROI params
DEFECT_CLASSES = (1, 2, 3)            # missing, extra_atom, extra_interstitial
ROI_MARGIN = 20
MIN_ROI_SIDE = 24
FALLBACK = "center"                   # "center" or "full"

# DINO normalization (ImageNet)
IMNET_MEAN = (0.485, 0.456, 0.406)
IMNET_STD  = (0.229, 0.224, 0.225)

# ----------------------------
# Load DINOv2
# ----------------------------
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()

# ----------------------------
# ROI helpers
# ----------------------------
def normalize01(img2d):
    img2d = img2d.astype(np.float32)
    p1, p99 = np.percentile(img2d, (1, 99))
    return np.clip((img2d - p1) / (p99 - p1 + 1e-6), 0, 1)

def clip_box(y0, x0, y1, x1, H, W):
    y0 = max(0, int(y0)); x0 = max(0, int(x0))
    y1 = min(H, int(y1)); x1 = min(W, int(x1))
    if y1 <= y0: y1 = min(H, y0 + 1)
    if x1 <= x0: x1 = min(W, x0 + 1)
    return y0, x0, y1, x1

def ensure_min_side(y0, x0, y1, x1, H, W, min_side=24):
    h = y1 - y0
    w = x1 - x0
    if h >= min_side and w >= min_side:
        return y0, x0, y1, x1
    cy = (y0 + y1) // 2
    cx = (x0 + x1) // 2
    half = max(min_side // 2, max(h, w) // 2)
    return clip_box(cy-half, cx-half, cy+half, cx+half, H, W)

def center_box(H, W, side=None):
    if side is None:
        side = min(H, W)
    side = min(side, H, W)
    y0 = (H - side) // 2
    x0 = (W - side) // 2
    return y0, x0, y0 + side, x0 + side

def roi_from_mask(img2d, seg, margin=20, min_side=24, defect_classes=(1,2,3), fallback="center"):
    """
    Returns:
      roi_uint8_rgb: HxWx3 uint8
      box: (y0,x0,y1,x1)
      used_defect_roi: bool
    """
    H, W = img2d.shape
    m = np.isin(seg, defect_classes)
    if np.any(m):
        ys, xs = np.where(m)
        y0 = ys.min() - margin
        y1 = ys.max() + margin + 1
        x0 = xs.min() - margin
        x1 = xs.max() + margin + 1
        y0, x0, y1, x1 = clip_box(y0, x0, y1, x1, H, W)
        y0, x0, y1, x1 = ensure_min_side(y0, x0, y1, x1, H, W, min_side=min_side)
        crop = normalize01(img2d[y0:y1, x0:x1])
        crop_u8 = (crop * 255.0 + 0.5).astype(np.uint8)
        roi = np.repeat(crop_u8[..., None], 3, axis=-1)  # RGB uint8
        return roi, (y0, x0, y1, x1), True

    # fallback
    if fallback == "full":
        y0, x0, y1, x1 = 0, 0, H, W
    else:
        y0, x0, y1, x1 = center_box(H, W, side=min(H, W))
    crop = normalize01(img2d[y0:y1, x0:x1])
    crop_u8 = (crop * 255.0 + 0.5).astype(np.uint8)
    roi = np.repeat(crop_u8[..., None], 3, axis=-1)
    return roi, (y0, x0, y1, x1), False

# ----------------------------
# Torch preprocessing WITHOUT numpy bridge
# ----------------------------
def pil_to_tensor_no_numpy(pil_img: Image.Image) -> torch.Tensor:
    """
    Returns float tensor CHW in [0,1] using ByteStorage.from_buffer (no torch.from_numpy).
    """
    pil_img = pil_img.convert("RGB")
    w, h = pil_img.size
    b = pil_img.tobytes()  # bytes
    storage = torch.ByteStorage.from_buffer(b)
    x = torch.ByteTensor(storage).view(h, w, 3).permute(2, 0, 1).contiguous()
    return x.float().div_(255.0)

def preprocess_rgb_u8_batch(rgb_u8_list, out_size=224, device="cuda"):
    """
    rgb_u8_list: list of HxWx3 uint8 numpy arrays
    Returns: torch tensor (B,3,out_size,out_size) normalized for DINO
    """
    tensors = []
    for arr in rgb_u8_list:
        # build PIL without torch/torchvision tensor conversion
        h, w, _ = arr.shape
        pil = Image.frombytes("RGB", (w, h), arr.tobytes())
        t = pil_to_tensor_no_numpy(pil)  # (3,H,W) float
        tensors.append(t)

    x = torch.stack(tensors, dim=0)  # CPU float
    # resize to PATCH_SIZE using torch only
    x = F.interpolate(x, size=(out_size, out_size), mode="bicubic", align_corners=False)

    # normalize
    mean = torch.tensor(IMNET_MEAN, dtype=x.dtype).view(1, 3, 1, 1)
    std  = torch.tensor(IMNET_STD,  dtype=x.dtype).view(1, 3, 1, 1)
    x = (x - mean) / std

    return x.to(device, non_blocking=True)

@torch.inference_mode()
def embed_batch_rgb_u8(rgb_u8_list):
    batch = preprocess_rgb_u8_batch(rgb_u8_list, out_size=PATCH_SIZE, device=DEVICE)
    feats = model.forward_features(batch)["x_norm_patchtokens"]
    emb = feats.mean(dim=1).cpu().numpy()
    return emb

# ----------------------------
# Main: compute ROI embeddings
# ----------------------------
with h5py.File(h5_in, "r") as fin, h5py.File(h5_masks, "r") as fmask, h5py.File(h5_out, "w") as fout:
    imgs = fin["images"]
    segs = fmask["seg"]
    labels = fin["labels"][:] if "labels" in fin else None
    n = len(imgs)

    emb_dim = 768  # dinov2_vitb14
    emb_ds = fout.create_dataset("embeddings", shape=(n, emb_dim), dtype="float32", compression="gzip")

    if labels is not None:
        fout.create_dataset("labels", data=labels)

    roi_box_ds = fout.create_dataset("roi_box_y0x0y1x1", shape=(n, 4), dtype="int32")
    roi_used_ds = fout.create_dataset("roi_used_defect", shape=(n,), dtype="uint8")

    for k in ["meta/paths", "meta/label_names"]:
        if k in fin:
            fout.create_dataset(k, data=fin[k][:])

    buf = []
    idxs = []

    for i in range(n):
        img2d = imgs[i]
        seg = segs[i]

        roi_rgb_u8, box, used = roi_from_mask(
            img2d, seg,
            margin=ROI_MARGIN,
            min_side=MIN_ROI_SIDE,
            defect_classes=DEFECT_CLASSES,
            fallback=FALLBACK
        )

        roi_box_ds[i] = np.array(box, dtype=np.int32)
        roi_used_ds[i] = np.uint8(1 if used else 0)

        buf.append(roi_rgb_u8)
        idxs.append(i)

        if len(buf) == batch_size or i == n - 1:
            emb = embed_batch_rgb_u8(buf)
            emb_ds[idxs[0]:idxs[0] + len(buf)] = emb
            buf, idxs = [], []

print("Saved ROI embeddings to", h5_out)

# ----------------------------
# Quick sanity stats
# ----------------------------
with h5py.File(h5_out, "r") as f:
    emb = f["embeddings"][:]
    used = f["roi_used_defect"][:]
    labels = f["labels"][:] if "labels" in f else None
    names = [x.decode() for x in f["meta/label_names"][:]] if "meta/label_names" in f else None

print("ROI used (defect-based) fraction:", float(used.mean()))
if labels is not None and names is not None:
    for lid in np.unique(labels):
        m = emb[labels == lid].mean()
        s = emb[labels == lid].std()
        frac = float(used[labels == lid].mean())
        print(f"{names[int(lid)]:>20s} | emb mean {m:.4f} std {s:.4f} | defect-ROI frac {frac:.3f}")


In [ ]:
# Jupyter cell: MASK-FREE DINOv2 embeddings (top-k patch pooling) + analysis (UMAP + top-k sweep) + optional ROI vs no-mask comparison
# - No masks/ROI required
# - Avoids torch->numpy bridge (RuntimeError: Numpy is not available) by using .tolist() conversion
# - Produces: sto_embeddings_nomask_k32.h5 (and optionally multiple ks)
# - Analysis:
#   (A) Per-class embedding stats (if labels + meta/label_names exist)
#   (B) UMAP 2D visualization (uses umap-learn; will fall back to PCA if missing)
#   (C) Top-k sensitivity sweep: k in {8,16,32,64}
#   (D) ROI vs no-mask UMAP comparison if sto_embeddings_roi.h5 exists

import os, h5py, numpy as np, torch
import torch.nn.functional as F
from PIL import Image

# ----------------------------
# Config
# ----------------------------
DINO_MODEL = "dinov2_vitb14"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PATCH_SIZE = 224
batch_size = 64  # lower to reduce CPU <-> GPU overhead and .tolist() cost

h5_in = "sto_dataset.h5"

# main output (single-k run)
TOPK_MAIN = 32
h5_out_main = f"sto_embeddings_nomask_k{TOPK_MAIN}.h5"

# sweep settings (set to None to disable)
TOPK_SWEEP = [8, 16, 32, 64]   # comment out to disable sweep
DO_SWEEP = True

# optional comparison file (if it exists)
h5_roi = "sto_embeddings_roi.h5"
DO_ROI_COMPARE = os.path.exists(h5_roi)

# If you want to downsample for faster UMAP visualization
MAX_POINTS_FOR_UMAP = 4000

# DINO normalization (ImageNet)
IMNET_MEAN = (0.485, 0.456, 0.406)
IMNET_STD  = (0.229, 0.224, 0.225)

# ----------------------------
# Load DINOv2
# ----------------------------
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()

# ----------------------------
# Helpers
# ----------------------------
def normalize01(img2d):
    img2d = img2d.astype(np.float32)
    p1, p99 = np.percentile(img2d, (1, 99))
    return np.clip((img2d - p1) / (p99 - p1 + 1e-6), 0, 1)

def pil_to_tensor_no_numpy(pil_img: Image.Image) -> torch.Tensor:
    # returns float tensor CHW in [0,1] using ByteStorage.from_buffer (no torch.from_numpy)
    pil_img = pil_img.convert("RGB")
    w, h = pil_img.size
    b = pil_img.tobytes()
    storage = torch.ByteStorage.from_buffer(b)
    x = torch.ByteTensor(storage).view(h, w, 3).permute(2, 0, 1).contiguous()
    return x.float().div_(255.0)

def preprocess_gray2d_to_dino(img2d, out_size=224):
    g01 = normalize01(img2d)
    u8 = (g01 * 255.0 + 0.5).astype(np.uint8)
    rgb = np.repeat(u8[..., None], 3, axis=-1)  # HxWx3 uint8

    h, w, _ = rgb.shape
    pil = Image.frombytes("RGB", (w, h), rgb.tobytes())
    x = pil_to_tensor_no_numpy(pil).unsqueeze(0)  # (1,3,H,W)

    x = F.interpolate(x, size=(out_size, out_size), mode="bicubic", align_corners=False)

    mean = torch.tensor(IMNET_MEAN, dtype=x.dtype).view(1, 3, 1, 1)
    std  = torch.tensor(IMNET_STD,  dtype=x.dtype).view(1, 3, 1, 1)
    x = (x - mean) / std
    return x

@torch.inference_mode()
def embed_batch_topk_torch(img2d_list, topk=32):
    xs = [preprocess_gray2d_to_dino(im, out_size=PATCH_SIZE) for im in img2d_list]
    x = torch.cat(xs, dim=0).to(DEVICE, non_blocking=True)  # (B,3,224,224)

    feats = model.forward_features(x)["x_norm_patchtokens"]  # (B, N, D)
    score = torch.linalg.norm(feats, dim=-1)                # (B, N)
    k = min(int(topk), feats.shape[1])
    idx = torch.topk(score, k=k, dim=1, largest=True).indices  # (B, k)

    B, N, D = feats.shape
    idx_exp = idx.unsqueeze(-1).expand(B, k, D)
    top_tokens = torch.gather(feats, dim=1, index=idx_exp)  # (B,k,D)
    emb = top_tokens.mean(dim=1)                            # (B,D)

    return emb.detach().to("cpu", non_blocking=False).float().contiguous()

def write_embeddings_nomask(h5_out, topk):
    with h5py.File(h5_in, "r") as fin, h5py.File(h5_out, "w") as fout:
        imgs = fin["images"]
        labels = fin["labels"][:] if "labels" in fin else None
        n = len(imgs)

        emb_dim = 768  # vitb14
        emb_ds = fout.create_dataset("embeddings", shape=(n, emb_dim), dtype="float32", compression="gzip")
        if labels is not None:
            fout.create_dataset("labels", data=labels)

        for k in ["meta/paths", "meta/label_names"]:
            if k in fin:
                fout.create_dataset(k, data=fin[k][:])

        buf, idxs = [], []
        for i in range(n):
            buf.append(imgs[i])
            idxs.append(i)
            if len(buf) == batch_size or i == n - 1:
                emb_t = embed_batch_topk_torch(buf, topk=topk)               # torch CPU tensor
                emb_np = np.asarray(emb_t.tolist(), dtype=np.float32)        # no torch->numpy bridge
                emb_ds[idxs[0]:idxs[0] + len(buf)] = emb_np
                buf, idxs = [], []
    print(f"[OK] Saved mask-free embeddings: {h5_out} (topk={topk})")

def load_embeddings(h5_path):
    with h5py.File(h5_path, "r") as f:
        emb = f["embeddings"][:]
        labels = f["labels"][:] if "labels" in f else None
        names = [x.decode() for x in f["meta/label_names"][:]] if "meta/label_names" in f else None
    return emb, labels, names

def per_class_stats(emb, labels, names, title=""):
    if title:
        print("\n" + title)
    print("Embeddings shape:", emb.shape, "dtype:", emb.dtype)
    print("Global mean:", float(emb.mean()), "Global std:", float(emb.std()))
    if labels is None or names is None:
        print("No labels and/or label_names found -> skipping per-class stats.")
        return
    print("\nPer-class stats:")
    for lid in np.unique(labels):
        m = float(emb[labels == lid].mean())
        s = float(emb[labels == lid].std())
        n = int((labels == lid).sum())
        print(f"{names[int(lid)]:>20s} | n={n:5d} | mean={m:.4f} | std={s:.4f}")

def sample_for_viz(X, y=None, max_points=4000, seed=0):
    n = len(X)
    if n <= max_points:
        return X, y
    rng = np.random.default_rng(seed)
    idx = rng.choice(n, size=max_points, replace=False)
    return X[idx], (y[idx] if y is not None else None)

def umap_or_pca_2d(X, seed=0):
    # Try UMAP; if unavailable, fall back to PCA
    try:
        import umap
        reducer = umap.UMAP(n_components=2, random_state=seed, n_neighbors=30, min_dist=0.05)
        Z = reducer.fit_transform(X)
        method = "UMAP"
        return Z, method
    except Exception as e:
        from sklearn.decomposition import PCA
        Z = PCA(n_components=2, random_state=seed).fit_transform(X)
        method = "PCA (fallback; install umap-learn for UMAP)"
        return Z, method

def plot_2d(Z, labels=None, names=None, title="2D embedding"):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7, 6))
    if labels is None:
        plt.scatter(Z[:, 0], Z[:, 1], s=4)
    else:
        # plot each class separately for readability
        for lid in np.unique(labels):
            m = labels == lid
            lab = names[int(lid)] if names is not None and int(lid) < len(names) else str(int(lid))
            plt.scatter(Z[m, 0], Z[m, 1], s=4, label=lab)
        plt.legend(markerscale=3, fontsize=8)
    plt.title(title)
    plt.xlabel("dim-1")
    plt.ylabel("dim-2")
    plt.tight_layout()
    plt.show()

def simple_separation_score(Z, labels):
    # quick numeric proxy: ratio of between-class centroid variance to within-class variance
    if labels is None:
        return np.nan
    labs = np.unique(labels)
    centroids = []
    within = []
    for lid in labs:
        Xc = Z[labels == lid]
        centroids.append(Xc.mean(axis=0))
        within.append(((Xc - Xc.mean(axis=0))**2).sum(axis=1).mean())
    centroids = np.stack(centroids, axis=0)
    between = ((centroids - centroids.mean(axis=0))**2).sum(axis=1).mean()
    within = float(np.mean(within))
    return float(between / (within + 1e-12))

# ----------------------------
# 1) Generate embeddings (single-k)
# ----------------------------
write_embeddings_nomask(h5_out_main, TOPK_MAIN)

# ----------------------------
# 2) Sanity stats + UMAP/PCA plot for main file
# ----------------------------
emb, labels, names = load_embeddings(h5_out_main)
per_class_stats(emb, labels, names, title=f"=== MASK-FREE Embeddings (topk={TOPK_MAIN}) ===")

Xviz, yviz = sample_for_viz(emb, labels, max_points=MAX_POINTS_FOR_UMAP, seed=0)
Z, method = umap_or_pca_2d(Xviz, seed=0)
score = simple_separation_score(Z, yviz) if yviz is not None else np.nan
plot_2d(Z, yviz, names, title=f"{method} of MASK-FREE embeddings (topk={TOPK_MAIN}) | sep={score:.3f}")

# ----------------------------
# 3) Top-k sensitivity sweep (optional)
#    - regenerates embeddings for each k
#    - computes UMAP/PCA sep score (quick proxy)
# ----------------------------
if DO_SWEEP and TOPK_SWEEP is not None:
    print("\n=== TOP-K SENSITIVITY SWEEP ===")
    sweep_results = []
    for k in TOPK_SWEEP:
        outk = f"sto_embeddings_nomask_k{k}.h5"
        if not os.path.exists(outk):
            write_embeddings_nomask(outk, k)
        Ek, yk, nk = load_embeddings(outk)

        Xk, yk_v = sample_for_viz(Ek, yk, max_points=MAX_POINTS_FOR_UMAP, seed=0)
        Zk, methodk = umap_or_pca_2d(Xk, seed=0)
        sepk = simple_separation_score(Zk, yk_v) if yk_v is not None else np.nan
        sweep_results.append((k, sepk, methodk))
        print(f"topk={k:>3d} | {methodk:<40s} | sep={sepk:.4f}")

    # plot sweep curve
    import matplotlib.pyplot as plt
    ks = [r[0] for r in sweep_results]
    ss = [r[1] for r in sweep_results]
    plt.figure(figsize=(6,4))
    plt.plot(ks, ss, marker="o")
    plt.title("Top-k sweep: separation proxy (higher is better)")
    plt.xlabel("topk")
    plt.ylabel("between/within (2D) proxy")
    plt.tight_layout()
    plt.show()

# ----------------------------
# 4) ROI vs No-mask comparison (optional)
#    - requires sto_embeddings_roi.h5 existing
# ----------------------------
if DO_ROI_COMPARE:
    print("\n=== ROI vs NO-MASK COMPARISON ===")
    emb_roi, lab_roi, names_roi = load_embeddings(h5_roi)

    # sample same number of points for fair-ish comparison
    max_pts = min(MAX_POINTS_FOR_UMAP, len(emb), len(emb_roi))
    Xn, yn = sample_for_viz(emb, labels, max_points=max_pts, seed=1)
    Xr, yr = sample_for_viz(emb_roi, lab_roi, max_points=max_pts, seed=1)

    Zn, mn = umap_or_pca_2d(Xn, seed=1)
    Zr, mr = umap_or_pca_2d(Xr, seed=1)

    sn = simple_separation_score(Zn, yn) if yn is not None else np.nan
    sr = simple_separation_score(Zr, yr) if yr is not None else np.nan

    plot_2d(Zn, yn, names, title=f"{mn} NO-MASK (topk={TOPK_MAIN}) | sep={sn:.3f}")
    plot_2d(Zr, yr, names_roi, title=f"{mr} ROI-based | sep={sr:.3f}")

    print(f"NO-MASK sep={sn:.4f} | ROI sep={sr:.4f}")
else:
    print("\n[INFO] ROI comparison skipped (sto_embeddings_roi.h5 not found).")


In [ ]:
import h5py, numpy as np

EMB_H5 = "sto_embeddings_nomask_k32.h5"   # you already generated this
# (works also for sto_embeddings_roi.h5 if you want to compare)

with h5py.File(EMB_H5, "r") as f:
    X = f["embeddings"][:]                  # (N,768) float32
    y_true = f["labels"][:] if "labels" in f else None
    names = None
    if "meta/label_names" in f:
        names = [x.decode() for x in f["meta/label_names"][:]]

print("X:", X.shape, X.dtype)
print("labels:", None if y_true is None else y_true.shape)
print("names:", names)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler(with_mean=True, with_std=True)
Xz = scaler.fit_transform(X)

# Optional: keep enough dims for clustering stability (often helps)
PCA_DENOISE = 128
pca0 = PCA(n_components=min(PCA_DENOISE, Xz.shape[1]), random_state=0)
X0 = pca0.fit_transform(Xz)

print("X0:", X0.shape, "explained:", float(pca0.explained_variance_ratio_.sum()))


In [ ]:
import numpy as np

def corr_filter(X, thr=0.95):
    # X: (N,D)
    C = np.corrcoef(X, rowvar=False)
    C = np.nan_to_num(C, nan=0.0, posinf=0.0, neginf=0.0)
    keep = []
    removed = np.zeros(C.shape[0], dtype=bool)
    for i in range(C.shape[0]):
        if removed[i]:
            continue
        keep.append(i)
        # remove highly correlated future dims
        j = np.where(np.abs(C[i]) > thr)[0]
        j = j[j > i]
        removed[j] = True
    return np.array(keep, dtype=int)

keep_corr = corr_filter(X0, thr=0.95)
X1 = X0[:, keep_corr]
print("after corr filter:", X1.shape, "kept:", len(keep_corr))


In [ ]:
from sklearn.cluster import KMeans

def feature_separation_proxy(X, k=4, seed=0):
    km = KMeans(n_clusters=k, n_init=10, random_state=seed).fit(X)
    labels = km.labels_
    mu = km.cluster_centers_
    # between-class variance per dim
    between = mu.var(axis=0)
    # within-class variance per dim
    within = np.zeros(X.shape[1], dtype=np.float64)
    for c in range(k):
        xc = X[labels == c]
        if len(xc) > 1:
            within += xc.var(axis=0)
    within /= max(1, k)
    score = between / (within + 1e-8)
    return score

proxy = feature_separation_proxy(X1, k=4, seed=0)
order = np.argsort(proxy)[::-1]

# keep top dims
TOP_DIMS = min(64, X1.shape[1])
keep_sep = order[:TOP_DIMS]
X2 = X1[:, keep_sep]
print("after sep proxy:", X2.shape)


In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0.10)
X3 = vt.fit_transform(X2)
print("after variance threshold:", X3.shape)


In [ ]:
from sklearn.metrics import silhouette_score

# PCA for final clustering space
PC_FINAL = min(32, X3.shape[1])
pca = PCA(n_components=PC_FINAL, random_state=0)
Z = pca.fit_transform(X3)
print("Z:", Z.shape, "explained:", float(pca.explained_variance_ratio_.sum()))

Ks = list(range(2, 9))
scores = []
models = []
for k in Ks:
    km = KMeans(n_clusters=k, n_init=20, random_state=0).fit(Z)
    s = silhouette_score(Z, km.labels_)
    scores.append(s)
    models.append(km)
    print(f"k={k:2d}  silhouette={s:.4f}")

best_i = int(np.argmax(scores))
best_k = Ks[best_i]
best_km = models[best_i]
pred = best_km.labels_
print("\nBEST k =", best_k, "silhouette =", scores[best_i])


In [ ]:
import matplotlib.pyplot as plt

# t-SNE
from sklearn.manifold import TSNE
ts = TSNE(n_components=2, perplexity=30, init="pca", learning_rate="auto", random_state=0)
Y = ts.fit_transform(Z)

plt.figure(figsize=(7,6))
plt.scatter(Y[:,0], Y[:,1], c=pred, s=8, cmap="tab10")
plt.title(f"t-SNE of DINO embeddings (k={best_k})")
plt.axis("off")
plt.show()

# Optional: if true labels exist, show cluster-vs-label table
if y_true is not None and names is not None:
    import pandas as pd
    C = np.zeros((best_k, len(names)), dtype=int)
    for i in range(len(pred)):
        C[pred[i], int(y_true[i])] += 1
    df = pd.DataFrame(C, columns=names)
    display(df)


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ---- assumes you already have:
# X (N,768), y_true (N,), names (list), and Z (N,32) from your previous run

# 1) compute bulk centroid in ORIGINAL embedding space (X)
bulk_id = names.index("Bulk")
X_bulk = X[y_true == bulk_id].astype(np.float32)

# L2 normalize for cosine
def l2norm_rows(A, eps=1e-8):
    n = np.linalg.norm(A, axis=1, keepdims=True)
    return A / (n + eps)

Xn = l2norm_rows(X.astype(np.float32))
bulk_centroid = Xn[y_true == bulk_id].mean(axis=0)
bulk_centroid = bulk_centroid / (np.linalg.norm(bulk_centroid) + 1e-8)

# 2) anomaly score = cosine distance to bulk centroid
cos_sim = (Xn * bulk_centroid[None, :]).sum(axis=1)          # in [-1,1]
anomaly = (1.0 - cos_sim).astype(np.float32)                 # in [0,2] approx

print("anomaly stats:", float(anomaly.min()), float(anomaly.mean()), float(anomaly.max()))

# 3) append anomaly to Z (your PCA space)
Z_aug = np.concatenate([Z, anomaly[:, None]], axis=1).astype(np.float32)
print("Z_aug:", Z_aug.shape)

# 4) re-run k sweep
Ks = list(range(2, 9))
scores = []
models = []
for k in Ks:
    km = KMeans(n_clusters=k, n_init=30, random_state=0).fit(Z_aug)
    s = silhouette_score(Z_aug, km.labels_)
    scores.append(s)
    models.append(km)
    print(f"k={k:2d}  silhouette={s:.4f}")

best_i = int(np.argmax(scores))
best_k = Ks[best_i]
pred_aug = models[best_i].labels_
print("\nBEST k =", best_k, "silhouette =", scores[best_i])


In [ ]:
import pandas as pd
import numpy as np

C = np.zeros((best_k, len(names)), dtype=int)
for i in range(len(pred_aug)):
    C[pred_aug[i], int(y_true[i])] += 1

df = pd.DataFrame(C, columns=names)
display(df)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

ts = TSNE(n_components=2, perplexity=30, init="pca", learning_rate="auto", random_state=0)
Y = ts.fit_transform(Z_aug)

plt.figure(figsize=(7,6))
plt.scatter(Y[:,0], Y[:,1], c=pred_aug, s=8, cmap="tab10")
plt.title(f"t-SNE of DINO embeddings + bulk-anomaly (k={best_k})")
plt.axis("off")
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

cl = AgglomerativeClustering(
    n_clusters=4,
    metric="cosine",
    linkage="average"
)
pred_cos = cl.fit_predict(Z_aug)

print("silhouette (cosine agglo):",
      silhouette_score(Z_aug, pred_cos, metric="cosine"))
